<h1>Refining Brazil's Mortality Data: A Guide to Transforming SIM-DATASUS Data with Standardization to Control for Confounding</h1>

After downloading data from the Mortality Information System (SIM-DATASUS, see [Unlocking Brazil's Mortality Data: A Guide to Extracting SIM-DATASUS Data with the microdatasus Package in R](https://medium.com/@victorfoscarini/unlocking-brazils-mortality-data-a-guide-to-extracting-sim-datasus-data-with-the-microdatasus-d7ea7bb1cc61)), we want to preprocess it for analysis. The datasets come yearly for the state. We group them with each .csv file containing all data from 1996 to 2022 for a state. The reason we chose this year range is because since 1996 SIM has been using [ICD-10](https://icd.who.int/browse10/2019/en) (the 10th version of the International Classification of Diseases) and 2022 is the last year available at the moment I write. Although [ICD-11](https://icd.who.int/browse/2024-01/mms/en) is already available, they are still using ICD-10, which is great for us that want to analyze data since while there are dictionaries to translate codes of an older to a newer ICD version, the translation is a very complicated many-to-many function.

SIM-DATASUS datasets also contain a lot of variables, but many of them are mostly null values. We use the [variable dictionary](http://vigilancia.saude.mg.gov.br/index.php/download/dicionario-de-dados-da-tabela-do-dasisms/?wpdmdl=630
) to understand it and select the variables that are important for our analysis. We'll then preprocess the subset of the dataset containing the chosen variables, deal with missing variables, and perform age/sex standardization, which is imperative for comparing mortality in different states in a heterogeneous country like Brazil. The reason for the importance of standardization is that mortality rates vary a lot according to demography (e.g. the older a population, the higher the mortality rate for diseases of the circulatory system). To perform that standardization, we get demographic data from [IBGE](https://www.ibge.gov.br/) (Brazilian Institute of Geography and Statistics), both for the [National Census](https://sidra.ibge.gov.br/pesquisa/censo-demografico) and [Population Estimation](https://www.ibge.gov.br/estatisticas/sociais/populacao/9103-estimativas-de-populacao.html?=&t=downloads).

Note: I'm using an integrated Colab + Google Drive environment for this project, but this could be done similarly with a local Jupyter lab, Jupyter notebook, or even just a regular Python compiler.

## Importing Libraries and Defining States and Capitals

In [ ]:
# Setting the version of the libraries I'll use to avoid the code
# stopping to work in the future because of library updates
!pip install numpy==1.23.5
!pip install pandas==2.1.1
!pip install xlrd==2.0.1
!pip install matplotlib==3.7.1
!pip install seaborn==0.13.0
!pip install ipython==7.34.0

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

#print in bold
BOLD = '\033[1m'
END = '\033[0m'

#connect to google drive
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/My Drive/seasonal_mortality_hub/data/'

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Defining an array with the acronyms of Brazilian states - technically federative units,
# since the Federal District (DF) is not a state, but I'll use states for simplicity
states = np.array(['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG',
                   'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR',
                   'RS', 'SC', 'SE', 'SP', 'TO'])

In [ ]:
# Defining an array with the state capital's acronym, the name of the state,
# and the name of the capital of that state
capitals = """
AC,Acre,Rio Branco
AL,Alagoas,Maceió
AP,Amapá,Macapá
AM,Amazonas,Manaus
BA,Bahia,Salvador
CE,Ceará,Fortaleza
DF,Distrito Federal,Brasília
ES,Espírito Santo,Vitória
GO,Goiás,Goiânia
MA,Maranhão,São Luís
MT,Mato Grosso,Cuiabá
MS,Mato Grosso do Sul,Campo Grande
MG,Minas Gerais,Belo Horizonte
PA,Pará,Belém
PB,Paraíba,João Pessoa
PR,Paraná,Curitiba
PE,Pernambuco,Recife
PI,Piauí,Teresina
RJ,Rio de Janeiro,Rio de Janeiro
RN,Rio Grande do Norte,Natal
RS,Rio Grande do Sul,Porto Alegre
RO,Rondônia,Porto Velho
RR,Roraima,Boa Vista
SC,Santa Catarina,Florianópolis
SP,São Paulo,São Paulo
SE,Sergipe,Aracaju
TO,Tocantins,Palmas
"""
capitals = [pd.Series(line.split(',')) for line in capitals.split('\n')[1:-1]]
capitals = pd.DataFrame(capitals)
capitals.columns = ['state', 'state_name', 'capital']
capitals = capitals.sort_values(by='state').reset_index().drop('index', axis=1)

## Preprocessing

### Grouping data

At first, I get the data I already downloaded from DATASUS using the R API (see [Unlocking Brazil's Mortality Data: A Guide to Extracting SIM-DATASUS Data with the microdatasus Package in R](https://medium.com/@victorfoscarini/unlocking-brazils-mortality-data-a-guide-to-extracting-sim-datasus-data-with-the-microdatasus-d7ea7bb1cc61)
), and group all datasets of each year by state into a single dataset containing all data from 1996 to 2022. I chose this year's range because before 1996, the ICD-9 was used instead of ICD-10, and 2022 was the last year available when I did this. It's also important to note that I only use data between 2000 and 2019 for many analyses. The problem with the data before 2000 is that there's too much missing data, and I chose to ignore the data after 2019 because I wanted to see the long-term pattern and not have to consider how it changed due to COVID-19.


In [ ]:
%%time
for state in states:
  print(BOLD+state+END)
  try:
    df = pd.read_csv(f'{path}sim/original_data/{state}/DO{state}{1996}.csv',
                     index_col=0, dtype=object)
  except:
    df = pd.read_csv(f'{path}sim/original_data/{state}/DO{state}{1996}.csv',
                     index_col=0, dtype=object, encoding='latin1')
  df.columns = [x.upper() for x in df.columns]
  for year in np.arange(1996+1,2022+1):
      try:
        data = pd.read_csv(f'{path}sim/original_data/{state}/DO{state}{year}.csv',
                           index_col=0, dtype=object)
      except:
        data = pd.read_csv(f'{path}sim/original_data/{state}/DO{state}{year}.csv',
                           index_col=0, dtype=object, encoding='latin1')
      data.columns = [x.upper() for x in data.columns]
      cols = []
      for col in data:
          if col in df:
              cols.append(col)
      df = pd.concat([df[cols], data[cols]])
  df.to_csv(f'{path}sim/states/sim_{state}_1996_2022.csv', index=False)

AC
AL
AM
AP
BA
CE
DF
ES
GO
MA
MG
MS
MT
PA
PB
PE
PI
PR
RJ
RN
RO
RR
RS
SC
SE
SP
TO
CPU times: user 13min 48s, sys: 4min, total: 17min 49s
Wall time: 26min 47s


In [ ]:
%%time

#checking data - death count by state
death_count = []

for state in states:
  data = pd.read_csv(f'{path}/sim/states/sim_{state}_1996_2022.csv', index_col=None)
  death_count.append({'STATE':state, 'DEATH_COUNT':data.shape[0]})

death_count = pd.DataFrame(death_count)
death_count.sort_values(by='DEATH_COUNT')

CPU times: user 1min 37s, sys: 33.2 s, total: 2min 10s
Wall time: 2min 19s


,STATE,DEATH_COUNT
21,RR,50598
3,AP,67647
0,AC,85983
26,TO,170336
20,RO,194243
6,DF,296286
24,SE,307120
2,AM,381978
11,MS,387953
12,MT,407550


### Preprocessing: states

**Data on population and demographics**

<br>

My idea is to use data from IBGE - census and population estimation. Data available:
- census: 2000, 2010, 2022
- pop est: 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2008, 2009, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021
- lacking both: 1996, 2007
- lacking pop est: 1996, 2007, 2010, 2022
- repeated: 2000 - census is more trustworthy; however, it might break the pattern due to having a different gathering strategy

I will use population estimation data to get the typical death rate in a city/state by year. For the years that lack data, I perform interpolation. Note that I could use data from the census for some of the missing years (not all, just 2000, 2010, and 2022). However, I chose not to because the census and the population estimation use different strategies, and mixing them could break the pattern.

What I use the census data for is getting the demographic pyramid to get the standardized rate of death in a city/state after adjustment, considering sex and age as possible confounding factors.

<br>

**Adjustment to Age and Sex**

<br>

Since age and sex are confounding factors, you cannot compare the crude mortality rate of two populations with different demographic structures. Therefore, I apply direct age/sex standardization to get a state's age/sex-adjusted mortality rates.

That adjustment allows me to compare two states with different demographic structures. I use Brazil's demographics (according to IBGE's census) as the standard population for the adjustment.

With the data of number of deaths by age and sex (SIM) and the demographics of the city (population estimation + IBGEs census), I calculate the rate of death inside each age-sex bracket ([Male-Female] and [0-4,5-9,10-14,...,85-89,90-100,100+]) and then take the weighted average of that rates using the percentage of the Brazilian population inside each group as weights.

Arrays used on the Standardization Equation:
- PPAS = array containing Percentage of Population in each Age/Sex group i, i=(Male [0-4], Female [0-4], M [5-9], F [5-9] ..., M [100,...], F [100,...])
- Death_Count = array containing total Death Count in each Age group i
- Death_Rate = array containing total Deaths by 100k population (taken yearly) in each age group i - note that here we are just dividing the Death_Count by the total population of the city, not for each age group, so we are just saying how much of the total rate of deaths come from that group, regardless of the group's size.

Standardized death rate inside each age group for the state, done by adjusting the total deaths for the ratio of the state's mean percentage population of an age group to the state's percentage population of that age group so that we have the rate of deaths by the mean population and can directly compare age group's deaths:

$$Deaths\_Standardized\_State_{i} = Death\_Rate_{i} \times \frac{mean(PPAS\_State)}{PPAS\_State_{i}}$$

Standardized death rate obtained by adjusting the standardized count from the state for the ratio of Brazil's percentage population of that age group to Brazil's mean percentage population of an age group so that we can directly compare states' deaths:

$$Deaths\_Standardized\_BR_{i} = Deaths\_Standardized\_State_{i} \times \frac{PPAS\_BR_{i}}{mean(PPAS\_BR)}$$

Note: In the end, Deaths_Standardized_State is the rate of deaths by the mean percentage population of an age group, and Deaths_Standardized_BR is the rate of deaths in a state/group adjusted to the percentage population of that group in Brazil.

<br>

**Dealing with missing Age and Sex data**

<br>

While calculating the Count_Standardized_State, there's a lot of missing data on age and/or sex. I only drop data that lacks a date; for missing Age/Sex, I create a category called 'null' and calculate the population for that group based on the weighted mean of the available data. If an observation is missing both Age and Sex, the population is the weighted mean of all demographics, taking the death counts as weights; if missing age group and not sex, the similar weighted mean for that sex; and if missing sex and not age, the weighted mean for that age group. That approximation would be good if the missing data is similar to the not-missing data. However, if the pattern is different, the approximation is bad. Since there's not a lot of missing data anyway, it shouldn't make too much difference. After making all the joins, I changed the category 'null' back to actual null data (np.nan).

In [ ]:
%%time

#state names
state_dict = capitals[['state', 'state_name']].set_index('state').to_dict()['state_name']

#demographic pyramid
demo_uf_ = pd.read_excel(f'{path}ibge/censo/piramide_demografica_uf_2010.xlsx', sheet_name=1, header=2, skiprows=3)
demo_uf_ = demo_uf_.fillna(method='ffill')
demo_uf_ = demo_uf_.iloc[:-1,[0,1,2,3]]
demo_uf_.columns = ['STATE', 'AGE', 'Male', 'Female']
demo_uf_ = pd.melt(demo_uf_, id_vars=['STATE', 'AGE'], value_vars=['Male', 'Female'],
                   var_name='SEX', value_name='POP_PERCENT')
demo_uf_['POP_PERCENT'] = demo_uf_['POP_PERCENT']*0.01

demo_br_ = pd.read_excel(f'{path}ibge/censo/piramide_demografica_br_2010.xlsx',
                          sheet_name=1, header=2, skiprows=3).iloc[:,[1,2,3]].dropna()
demo_br_ = demo_br_.fillna(method='ffill')
demo_br_.columns = ['AGE', 'Male', 'Female']
demo_br_ = pd.melt(demo_br_, id_vars=['AGE'],
                  value_vars=['Male', 'Female'])
demo_br_.columns = ['AGE', 'SEX', 'POP_PERCENT_BR']
demo_br_['POP_PERCENT_BR'] = demo_br_['POP_PERCENT_BR']*0.01
demo_br_['AGE']
demo_br_['AGE'] = np.hstack((np.array(pd.cut(np.append(np.arange(0,100,5), np.arange(100,110,10)),
                              bins=np.append(np.append(np.arange(0,100,5), np.arange(100,110,10)), 120),
                              right=False)),)*2).astype(object)
demo_br_ = demo_br_.sort_values(by=['AGE', 'SEX'], ascending=[True, False])

for state in states:
  state_name = state_dict[state]
  print(BOLD+f'{state} / {state_name}'+END)

  # taking the population of the state in the interval 1996-2022
  pop = {}
  for year in np.arange(1996,2013+1):
    try:
      data = pd.read_excel(f'{path}ibge/estimativas/pop_{year}.xls', sheet_name=0)
      data = data[data.iloc[:,0]==state]
      pop[year] = data.iloc[:,4].sum()
    except:
      pop[year] = np.nan

  for year in np.arange(2014,2022+1):
    try:
      data = pd.read_excel(f'{path}ibge/estimativas/pop_{year}.xls', sheet_name=1)
      data = data[data.iloc[:,0]==state]
      pop[year] = data.iloc[:,4].sum()
    except:
      pop[year] = np.nan
  pop = pd.DataFrame.from_dict(pop, orient='index').reset_index()
  pop.columns = ['year', 'pop']
  #if 2 or less numbers after point, drop all, if 3 drop just point and keep number
  pop['pop'] = pop['pop'].replace('\.\d{0,2}$','', regex=True).replace('\.', '', regex=True)
  #remove (\d) references and blank spaces
  pop['pop'] = pop['pop'].replace('\(\d\)', '', regex=True).replace(' ','', regex=True)
  pop['pop'] = pop['pop'].astype(float)
  # pop = pop.interpolate().bfill()
  # I'm mostly interpolating the data linearly (polynomial spline of order 1)
  # but technically extrapolating for the year 1996 and 2022, so I need a bit of workaround
  pop_for = pop.interpolate('spline', order=1, limit_direction='forward')
  pop_back = pop.interpolate('spline', order=1, limit_direction='backward')
  pop_for.loc[(pop_for.year==1996)] = pop_back.loc[(pop_back.year==1996)]
  pop = pop_for.copy()
  pop['pop'] = pop['pop'].astype(int)

  #getting dataset from SIM
  df = pd.read_csv(f'{path}sim/states/sim_{state}_1996_2022.csv', dtype='object')
  # print('1', df.shape)
  df = df[['CAUSABAS', 'DTOBITO', 'IDADE', 'SEXO', 'RACACOR']]
  df = df.rename(columns={'CAUSABAS':'CAUSE', 'DTOBITO':'DT',
                          'IDADE':'AGE', 'SEXO':'SEX', 'RACACOR':'RACECOLOR'})
  # rows without date information - mostly only year of death - will return as nan
  # print(2, df.shape)
  # print('Expected change in size -> decrease') #removing missing date
  df['DT'] = pd.to_datetime(df.DT, format='%d%m%Y', errors='coerce')
  df = df.dropna(subset=['DT'])
  # print(3, df.shape)
  # display(df.isna().sum())
  def f(x):
    if 0<=x<400:
      return 0
    elif 400<=x<600:
      return x-400
    else: #999 or mistake
      return np.nan

  #preprocessing
  df.AGE = df.AGE.astype(float).apply(lambda x: f(x))
  df.AGE = pd.cut(df.AGE, bins=np.append(np.append(np.arange(0,100,5), np.arange(100,110,10)), 120), right=False).astype(object)
  df.RACECOLOR = df.RACECOLOR.replace({'1':'White', '2':'Black', '3':'Yellow', '4':'Brown', '5':'Indigenous', '9':np.nan})
  df.SEX = df.SEX.replace({'1':'Male', '2':'Female', '9':np.nan, '0':np.nan})
  # print(4, df.shape)
  # print('Expected change in size -> decrease') #grouping by categories
  df = df.groupby(by=['CAUSE', 'DT', 'AGE',	'SEX',	'RACECOLOR'], observed=True, dropna=False).size().reset_index()
  # print(5, df.shape)
  # display(df.isna().sum())
  df.columns = ['CAUSE', 'DT', 'AGE',	'SEX',	'RACECOLOR', 'DEATH_COUNT']
  #getting death rate (deths by 1e5=100k)
  df['year'] = df.DT.dt.year
  # print(6, df.shape)
  # print('Fixed unexpected change in size')
  df = df.merge(pop, left_on=['year'], right_on=['year'], how='left')
  # print(7, df.shape)
  df['DEATH_RATE'] = df['DEATH_COUNT'] * (1e5/df['pop'])
  df = df.drop(columns=['year', 'pop'])

  #getting info about state's demographic pyramid
  demo_uf = demo_uf_[demo_uf_['STATE'] == state_name]
  demo_uf['AGE'] = np.hstack((np.array(pd.cut(np.append(np.arange(0,100,5), np.arange(100,110,10)),
                              bins=np.append(np.append(np.arange(0,100,5), np.arange(100,110,10)), 120),
                              right=False)),)*2).astype(object)
  demo_uf = demo_uf.sort_values(by=['AGE', 'SEX'], ascending=[True, False])

  #getting info about brazil's demographic pyramid
  demo_br = demo_br_.copy()

  #getting mean value of age group percentage population
  mean_uf = 1/len(demo_uf['POP_PERCENT'])
  mean_br = 1/len(demo_br['POP_PERCENT_BR'])

  #However, now I'll set a new group of 'null' values with mean weight
  #so that I can merge them as well
  df['AGE'] = df['AGE'].fillna('null')
  df['SEX'] = df['SEX'].fillna('null')

  # Algorithm for finding null demographics of deceased according to the weighted average of demographics
  # using DEATH_COUNTS as weights instead of a simple average that only considers demographics of the living

  #getting original demographics values
  #getting death count values
  deaths_counts = df[(df.AGE!='null')&(df.SEX!='null')][['AGE', 'SEX', 'DEATH_COUNT']].groupby(by=['AGE', 'SEX']).sum()
  deaths_counts = (deaths_counts/deaths_counts.sum()).reset_index()

  ##for demo_uf
  #joining demo + death count values
  demo_uf = demo_uf.merge(deaths_counts, right_on=['AGE', 'SEX'], left_on=['AGE', 'SEX'], how='left')
  #getting null demographics values
  demo_uf_null_age = demo_uf.copy()
  demo_uf_null_age['AGE'] = 'null'
  demo_uf_null_sex = demo_uf.copy()
  demo_uf_null_sex['SEX'] = 'null'
  demo_uf_null_all = demo_uf.copy()
  demo_uf_null_all['AGE'] = 'null'
  demo_uf_null_all['SEX'] = 'null'
  demo_uf_null = pd.concat([demo_uf_null_age, demo_uf_null_sex, demo_uf_null_all])
  #grouping to obtain new POP_PERCENT
  def weighted_average(data):
    return (data['POP_PERCENT']*data['DEATH_COUNT']).sum() / data['DEATH_COUNT'].sum()
  demo_uf_null = demo_uf_null.groupby(by=['STATE', 'AGE', 'SEX']).apply(weighted_average).reset_index(name='POP_PERCENT')
  #final data
  demo_uf = pd.concat([demo_uf[['STATE', 'AGE', 'SEX', 'POP_PERCENT']], demo_uf_null])

  ##for demo_br
  #joining demo + death count values
  demo_br = demo_br.merge(deaths_counts, right_on=['AGE', 'SEX'], left_on=['AGE', 'SEX'], how='left')
  #getting null demographics values
  demo_br_null_age = demo_br.copy()
  demo_br_null_age['AGE'] = 'null'
  demo_br_null_sex = demo_br.copy()
  demo_br_null_sex['SEX'] = 'null'
  demo_br_null_all = demo_br.copy()
  demo_br_null_all['AGE'] = 'null'
  demo_br_null_all['SEX'] = 'null'
  demo_br_null = pd.concat([demo_br_null_age, demo_br_null_sex, demo_br_null_all])
  #grouping to obtain new POP_PERCENT
  def weighted_average(data):
    return (data['POP_PERCENT_BR']*data['DEATH_COUNT']).sum() / data['DEATH_COUNT'].sum()
  demo_br_null = demo_br_null.groupby(by=['AGE', 'SEX']).apply(weighted_average).reset_index(name='POP_PERCENT_BR')
  #final data
  demo_br = pd.concat([demo_br[['AGE', 'SEX', 'POP_PERCENT_BR']], demo_br_null])

  # df_before = df[df.RACECOLOR.isna()].reset_index(drop=True)
  # print(8, df.shape)
  df = df.merge(demo_uf, left_on=['AGE', 'SEX'], right_on=['AGE', 'SEX'], how='left')
  df = df.merge(demo_br, left_on=['AGE', 'SEX'], right_on=['AGE', 'SEX'], how='left')
  # print('Fixed unexpected change in size')
  # print(9, df.shape)
  # df_after = df[df.RACECOLOR.isna()].reset_index(drop=True)

  df['POP_PERCENT'] = df['POP_PERCENT'].replace(0, np.nan)
  df['POP_PERCENT_BR'] = df['POP_PERCENT_BR'].replace(0,np.nan)

  #count standardized is taking the city's demographics into consideration
  #while count_standardized_br is taking the brazilian demographics into consideration
  df['DEATHS_STANDARDIZED'] = df['DEATH_RATE']*(mean_uf/df['POP_PERCENT'])
  df['DEATHS_STANDARDIZED_BR'] = df['DEATHS_STANDARDIZED']*(df['POP_PERCENT_BR']/mean_br)
  df = df[['CAUSE', 'DT', 'AGE', 'SEX', 'RACECOLOR', 'DEATH_COUNT',
           'DEATH_RATE', 'DEATHS_STANDARDIZED', 'DEATHS_STANDARDIZED_BR']]

  df['DEATHS_STANDARDIZED'] = df['DEATHS_STANDARDIZED'].fillna(0)
  df['DEATHS_STANDARDIZED_BR'] = df['DEATHS_STANDARDIZED_BR'].fillna(0)

  df = df.groupby(by=['CAUSE', 'DT', 'AGE', 'SEX', 'RACECOLOR'], dropna=False).sum().reset_index()

  #now, I also have to replace the 'null' values I used for merging with np.nan
  df = df.replace('null', np.nan)

  # print(10, df.shape)
  # display(df.isna().sum())

  df.to_csv(f'{path}sim/states_standardized/{state}_standardized_1996_2022.csv', index=False)

AC / Acre
AL / Alagoas
AM / Amazonas
AP / Amapá
BA / Bahia
CE / Ceará
DF / Distrito Federal
ES / Espírito Santo
GO / Goiás
MA / Maranhão
MG / Minas Gerais
MS / Mato Grosso do Sul
MT / Mato Grosso
PA / Pará
PB / Paraíba
PE / Pernambuco
PI / Piauí
PR / Paraná
RJ / Rio de Janeiro
RN / Rio Grande do Norte
RO / Rondônia
RR / Roraima
RS / Rio Grande do Sul
SC / Santa Catarina
SE / Sergipe
SP / São Paulo
TO / Tocantins
CPU times: user 15min 12s, sys: 52.3 s, total: 16min 5s
Wall time: 17min 22s


In [ ]:
%%time

# Checking results: Neoplasms deaths in 2022
# Interesting fact: it's expected that more developed places will have more cancer deaths due to having higher cancer
# rates because cancer is less avoidable than other diseases that kill people in less developed places before they
# even grow old enough to have cancer - that's not to be mistaken with the lethality of cancer that's smaller in
# richer places due to access to better treatment and earlier diagnosis.
neoplasms = []

for state in states:
  data = pd.read_csv(f'{path}/sim/states_standardized/{state}_standardized_1996_2022.csv', index_col=None)
  data.DT = pd.to_datetime(data.DT)
  neoplasms_state = (data[(data.DT.dt.year==2022)&(data.CAUSE>'C00')&(data.CAUSE<'D50')].sum(numeric_only=True)).to_dict()
  neoplasms_state['STATE'] = state
  neoplasms.append(neoplasms_state)

neoplasms = pd.DataFrame(neoplasms)[['STATE', 'DEATH_COUNT', 'DEATH_RATE', 'DEATHS_STANDARDIZED_BR']]
neoplasms.sort_values(by='DEATHS_STANDARDIZED_BR')

CPU times: user 54.8 s, sys: 5.07 s, total: 59.8 s
Wall time: 1min 6s


,STATE,DEATH_COUNT,DEATH_RATE,DEATHS_STANDARDIZED_BR
9,MA,4689.0,64.763413,79.596857
13,PA,5681.0,60.435405,86.847420
24,SE,1872.0,79.385071,92.828898
4,BA,14293.0,90.767256,93.592045
16,PI,3049.0,92.475205,94.255669
26,TO,1254.0,77.194028,94.673212
1,AL,2659.0,78.688218,94.688783
0,AC,558.0,60.699546,95.062850
14,PB,4471.0,109.569020,101.242663
15,PE,9595.0,99.093463,101.435347


### Preprocessing: state capitals

Similar to state, but done with state capital.

Note that here I select the CODMUNOCOR, which is where the death was registered, not CODMUNRES, which is where the deceased lives. That makes a considerable difference in analysing data from state capitals because of patient flow, that often happens from small cities to big cities.

In [ ]:
%%time

#state-capital dictionary
capital_dict = capitals[['state', 'capital']].set_index('state').to_dict()['capital']

#municipalities demographic pyramid
demo_muni_ = pd.read_excel(f'{path}ibge/censo/piramide_demografica_municipio_2010.xlsx', sheet_name=1, header=2, skiprows=3)
# demo_muni_ = demo_muni_.fillna(method='ffill')
demo_muni_ = demo_muni_.iloc[:-1,[0,1,2,3]]
demo_muni_.columns = ['CITY', 'AGE', 'Male', 'Female']
#only at the first line the city is written and after that we see null values
demo_muni_['CITY'] = demo_muni_['CITY'].fillna(method='ffill')
demo_muni_ = pd.melt(demo_muni_, id_vars=['CITY', 'AGE'], value_vars=['Male', 'Female'],
                   var_name='SEX', value_name='POP_PERCENT')
#cities with no population data have pop '...'
demo_muni_['POP_PERCENT'] = demo_muni_.POP_PERCENT.replace('...', np.nan)
demo_muni_ = demo_muni_.dropna()
#age groups with 0 population have pop '-'
#should leave as np.nan to avoid division by 0
demo_muni_['POP_PERCENT'] = demo_muni_.POP_PERCENT.replace('-', np.nan)
demo_muni_['POP_PERCENT'] = demo_muni_['POP_PERCENT']*0.01

#brazil demographic pyramid
demo_br_ = pd.read_excel(f'{path}ibge/censo/piramide_demografica_br_2010.xlsx',
                          sheet_name=1, header=2, skiprows=3).iloc[:,[1,2,3]].dropna()
demo_br_ = demo_br_.fillna(method='ffill')
demo_br_.columns = ['AGE', 'Male', 'Female']
demo_br_ = pd.melt(demo_br_, id_vars=['AGE'],
                   value_vars=['Male', 'Female'])
demo_br_.columns = ['AGE', 'SEX', 'POP_PERCENT_BR']
demo_br_['POP_PERCENT_BR'] = demo_br_['POP_PERCENT_BR']*0.01
demo_br_['AGE']
demo_br_['AGE'] = np.hstack((np.array(pd.cut(np.append(np.arange(0,100,5), np.arange(100,110,10)),
                              bins=np.append(np.append(np.arange(0,100,5), np.arange(100,110,10)), 120),
                              right=False)),)*2).astype(object)
demo_br_ = demo_br_.sort_values(by=['AGE', 'SEX'], ascending=[True, False])

for state in states:
  capital_name = capital_dict[state]
  print(BOLD+f'{capital_name} ({state})'+END)

  # taking the population of the state in the interval 1996-2022
  pop = {}
  for year in np.arange(1996,2013+1):
    try:
      data = pd.read_excel(f'{path}ibge/estimativas/pop_{year}.xls', sheet_name=0)
      data = data[(data.iloc[:,0]==state)&(data.iloc[:,3]==capital_name)]
      pop[year] = data.iloc[:,4].values[0]
    except:
      pop[year] = np.nan

  for year in np.arange(2014,2022+1):
    try:
      data = pd.read_excel(f'{path}ibge/estimativas/pop_{year}.xls', sheet_name=1)
      data = data[(data.iloc[:,0]==state)&(data.iloc[:,3]==capital_name)]
      pop[year] = data.iloc[:,4].values[0]
    except:
      pop[year] = np.nan

  pop = pd.DataFrame.from_dict(pop, orient='index').reset_index()
  pop.columns = ['year', 'pop']
  #if 2 or less numbers after point, drop all, if 3 drop just point and keep number
  pop['pop'] = pop['pop'].replace('\.\d{0,2}$','', regex=True).replace('\.', '', regex=True)
  #remove (\d) references and blank spaces
  pop['pop'] = pop['pop'].replace('\(\d\)', '', regex=True).replace(' ','', regex=True)
  pop['pop'] = pop['pop'].astype(float)
  # pop = pop.interpolate().bfill()
  # I'm mostly interpolating the data with linear regression (polynomial of order 1)
  # but technically extrapolating for the year 1996 and 2022, so I need a bit of workaround
  pop_for = pop.interpolate('spline', order=1, limit_direction='forward')
  pop_back = pop.interpolate('spline', order=1, limit_direction='backward')
  pop_for.loc[(pop_for.year==1996)] = pop_back.loc[(pop_back.year==1996)]
  pop = pop_for.copy()
  pop['pop'] = pop['pop'].astype(int)

  #getting dataset from SIM
  df = pd.read_csv(f'{path}sim/states/sim_{state}_1996_2022.csv', dtype='object')

  #selecting capital
  #capital code from IBGE data - cod.UF + cod.MUNIC
  data = pd.read_excel(f'{path}ibge/estimativas/pop_{2021}.xls', sheet_name=1)
  data = data[(data.iloc[:,0]==state)&(data.iloc[:,3]==capital_name)]
  capital_code = str(data.iloc[:,1].values[0])[:2]+str(data.iloc[:,2].values[0])[:4]
  #locate capital only using code from before
  df = df[df.CODMUNOCOR.str[:6]==capital_code]
  # print('1', df.shape)

  #selecting columns
  df = df[['CAUSABAS', 'DTOBITO', 'IDADE', 'SEXO', 'RACACOR']]
  df = df.rename(columns={'CAUSABAS':'CAUSE', 'DTOBITO':'DT',
                          'IDADE':'AGE', 'SEXO':'SEX', 'RACACOR':'RACECOLOR'})
  # rows without date information - mostly only year of death - will return as nan
  # print(2, df.shape)
  # print('Expected change in size -> decrease') #removing missing date
  df['DT'] = pd.to_datetime(df.DT, format='%d%m%Y', errors='coerce')
  df = df.dropna(subset=['DT'])
  # print(3, df.shape)
  # display(df.isna().sum())
  def f(x):
    if 0<=x<400:
      return 0
    elif 400<=x<600:
      return x-400
    else: #999 or mistake
      return np.nan

  #preprocessing
  df.AGE = df.AGE.astype(float).apply(lambda x: f(x))
  df.AGE = pd.cut(df.AGE, bins=np.append(np.append(np.arange(0,100,5), np.arange(100,110,10)), 120), right=False).astype(object)
  #df.RACECOLOR = df.RACECOLOR.replace({'1':'Branco', '2':'Preto', '3':'Amarelo', '4':'Pardo', '5':'Indígena', '9':np.nan})
  df.RACECOLOR = df.RACECOLOR.replace({'1':'White', '2':'Black', '3':'Yellow', '4':'Brown', '5':'Indigenous', '9':np.nan})
  df.SEX = df.SEX.replace({'1':'Male', '2':'Female', '9':np.nan, '0':np.nan})
  # print(4, df.shape)
  # print('Expected change in size -> decrease') #grouping by categories
  df = df.groupby(by=['CAUSE', 'DT', 'AGE',	'SEX',	'RACECOLOR'], observed=True, dropna=False).size().reset_index()
  # print(5, df.shape)
  # display(df.isna().sum())
  df.columns = ['CAUSE', 'DT', 'AGE',	'SEX',	'RACECOLOR', 'DEATH_COUNT']
  #getting death rate (deths by 1e5=100k)
  df['year'] = df.DT.dt.year
  # print(6, df.shape)
  # print('Fixed unexpected change in size')
  df = df.merge(pop, left_on=['year'], right_on=['year'], how='left')
  # print(7, df.shape)
  df['DEATH_RATE'] = df['DEATH_COUNT'] * (1e5/df['pop'])
  df = df.drop(columns=['year', 'pop'])

  #getting info about city's demographic pyramid
  demo_muni = demo_muni_[demo_muni_['CITY']==f'{capital_name} ({state})']
  demo_muni['AGE'] = np.hstack((np.array(pd.cut(np.append(np.arange(0,100,5), np.arange(100,110,10)),
                            bins=np.append(np.append(np.arange(0,100,5), np.arange(100,110,10)), 120),
                            right=False)),)*2).astype(object)
  demo_muni = demo_muni.sort_values(by=['AGE', 'SEX'], ascending=[True, False])

  #getting info about brazil's demographic pyramid
  demo_br = demo_br_.copy()

  #getting mean value of age group percentage population
  mean_muni = 1/len(demo_muni['POP_PERCENT'])
  mean_br = 1/len(demo_br['POP_PERCENT_BR'])

  # Algorithm for finding null demographics of deceased according to the weighted average of demographics
  # using DEATH_COUNTS as weights instead of a simple average that only considers demographics of the living

  #getting original demographics values
  #getting death count values
  deaths_counts = df[(df.AGE!='null')&(df.SEX!='null')][['AGE', 'SEX', 'DEATH_COUNT']].groupby(by=['AGE', 'SEX']).sum()
  deaths_counts = (deaths_counts/deaths_counts.sum()).reset_index()

  ##for demo_muni
  #joining demo + death count values
  demo_muni = demo_muni.merge(deaths_counts, right_on=['AGE', 'SEX'], left_on=['AGE', 'SEX'], how='left')
  #getting null demographics values
  demo_muni_null_age = demo_muni.copy()
  demo_muni_null_age['AGE'] = 'null'
  demo_muni_null_sex = demo_muni.copy()
  demo_muni_null_sex['SEX'] = 'null'
  demo_muni_null_all = demo_muni.copy()
  demo_muni_null_all['AGE'] = 'null'
  demo_muni_null_all['SEX'] = 'null'
  demo_muni_null = pd.concat([demo_muni_null_age, demo_muni_null_sex, demo_muni_null_all])
  #grouping to obtain new POP_PERCENT
  def weighted_average(data):
    return (data['POP_PERCENT']*data['DEATH_COUNT']).sum() / data['DEATH_COUNT'].sum()
  demo_muni_null = demo_muni_null.groupby(by=['CITY', 'AGE', 'SEX']).apply(weighted_average).reset_index(name='POP_PERCENT')
  #final data
  demo_muni = pd.concat([demo_muni[['CITY', 'AGE', 'SEX', 'POP_PERCENT']], demo_muni_null])

  ##for demo_br
  #joining demo + death count values
  demo_br = demo_br.merge(deaths_counts, right_on=['AGE', 'SEX'], left_on=['AGE', 'SEX'], how='left')
  #getting null demographics values
  demo_br_null_age = demo_br.copy()
  demo_br_null_age['AGE'] = 'null'
  demo_br_null_sex = demo_br.copy()
  demo_br_null_sex['SEX'] = 'null'
  demo_br_null_all = demo_br.copy()
  demo_br_null_all['AGE'] = 'null'
  demo_br_null_all['SEX'] = 'null'
  demo_br_null = pd.concat([demo_br_null_age, demo_br_null_sex, demo_br_null_all])
  #grouping to obtain new POP_PERCENT
  def weighted_average(data):
    return (data['POP_PERCENT_BR']*data['DEATH_COUNT']).sum() / data['DEATH_COUNT'].sum()
  demo_br_null = demo_br_null.groupby(by=['AGE', 'SEX']).apply(weighted_average).reset_index(name='POP_PERCENT_BR')
  #final data
  demo_br = pd.concat([demo_br[['AGE', 'SEX', 'POP_PERCENT_BR']], demo_br_null])

  df_after = df[df.RACECOLOR.isna()].reset_index(drop=True)
  # print(8, df.shape)
  df = df.merge(demo_muni, left_on=['AGE', 'SEX'], right_on=['AGE', 'SEX'], how='left')
  df = df.merge(demo_br, left_on=['AGE', 'SEX'], right_on=['AGE', 'SEX'], how='left')
  # print('Fixed unexpected change in size')
  # print(9, df.shape)
  df_before = df[df.RACECOLOR.isna()].reset_index(drop=True)

  df['POP_PERCENT'] = df['POP_PERCENT'].replace(0, np.nan)
  df['POP_PERCENT_BR'] = df['POP_PERCENT_BR'].replace(0,np.nan)

  #count standardized is taking the city's demographics into consideration
  #while count_standardized_br is taking the brazilian demographics into consideration
  df['DEATHS_STANDARDIZED'] = df['DEATH_RATE']*(mean_muni/df['POP_PERCENT'])
  df['DEATHS_STANDARDIZED_BR'] = df['DEATHS_STANDARDIZED']*(df['POP_PERCENT_BR']/mean_br)
  df = df[['CAUSE', 'DT', 'AGE', 'SEX', 'RACECOLOR', 'DEATH_COUNT',
           'DEATH_RATE', 'DEATHS_STANDARDIZED', 'DEATHS_STANDARDIZED_BR']]

  df['DEATHS_STANDARDIZED'] = df['DEATHS_STANDARDIZED'].fillna(0)
  df['DEATHS_STANDARDIZED_BR'] = df['DEATHS_STANDARDIZED_BR'].fillna(0)

  df = df.groupby(by=['CAUSE', 'DT', 'AGE', 'SEX', 'RACECOLOR'], dropna=False).sum().reset_index()

  #now, I also have to replace the 'null' values I used for merging with np.nan
  df = df.replace('null', np.nan)

  # print(10, df.shape)
  # display(df.isna().sum())

  df.to_csv(f'{path}sim/capitals_standardized/{state}_standardized_1996_2022.csv', index=False)

Rio Branco (AC)
Maceió (AL)
Manaus (AM)
Macapá (AP)
Salvador (BA)
Fortaleza (CE)
Brasília (DF)
Vitória (ES)
Goiânia (GO)
São Luís (MA)
Belo Horizonte (MG)
Campo Grande (MS)
Cuiabá (MT)
Belém (PA)
João Pessoa (PB)
Recife (PE)
Teresina (PI)
Curitiba (PR)
Rio de Janeiro (RJ)
Natal (RN)
Porto Velho (RO)
Boa Vista (RR)
Porto Alegre (RS)
Florianópolis (SC)
Aracaju (SE)
São Paulo (SP)
Palmas (TO)
CPU times: user 9min 45s, sys: 35.8 s, total: 10min 21s
Wall time: 11min 15s


In [ ]:
%%time

# Checking results: Neoplasms deaths in 2022
# Interesting fact: it's expected that more developed places will have more cancer deaths due to having higher cancer
# rates because cancer is less avoidable than other diseases that kill people in less developed places before they
# even grow old enough to have cancer - that's not to be mistaken with the lethality of cancer that's smaller in
# richer places due to access to better treatment and earlier diagnosis.
# Another interesting result is how some cities here have very high death rates, much higher then the state's rates,
# I'd say this is due to patient flow from small cities to the state capital.

neoplasms = []

for state in states:
  data = pd.read_csv(f'{path}/sim/capitals_standardized/{state}_standardized_1996_2022.csv', index_col=None)
  data.DT = pd.to_datetime(data.DT)
  neoplasms_state = (data[(data.DT.dt.year==2022)&(data.CAUSE>'C00')&(data.CAUSE<'D50')].sum(numeric_only=True)).to_dict()
  neoplasms_state['STATE'] = state
  neoplasms.append(neoplasms_state)

neoplasms = pd.DataFrame(neoplasms)[['STATE', 'DEATH_COUNT', 'DEATH_RATE', 'DEATHS_STANDARDIZED_BR']]
neoplasms.sort_values(by='DEATHS_STANDARDIZED_BR')

CPU times: user 18.9 s, sys: 745 ms, total: 19.7 s
Wall time: 22 s


,STATE,DEATH_COUNT,DEATH_RATE,DEATHS_STANDARDIZED_BR
6,DF,2752.0,87.825088,121.810376
3,AP,395.0,74.274368,136.379822
18,RJ,12142.0,178.472028,137.486224
0,AC,395.0,92.835017,141.604868
25,SP,19157.0,153.655356,141.822152
21,RR,390.0,85.992106,152.828216
2,AM,2293.0,100.033766,158.383934
11,MS,1478.0,159.626746,169.514505
1,AL,1469.0,141.544794,172.313057
5,CE,4321.0,158.850424,174.851413


## Checking total deaths

In [ ]:
# How many deaths are there between 2000 and 2019?
death_column = 'DEATH_COUNT'
df = None
for state in states:
  data = pd.read_csv(path+f'sim/states_standardized/{state}_standardized_1996_2022.csv')
  data = data[['DT', death_column]]
  data['DT'] = pd.to_datetime(data['DT'])
  data = data[(data.DT.dt.year>=2000)&(data.DT.dt.year<=2019)]
  data['STATE'] = state
  if df is None:
    df = data
  else:
    df = pd.concat([df, data])
# I group all data from 2000-2019 in a single year that I set as
# 2024 only to be able to perform operations with it
df.DT = df.DT.dt.strftime('%m-%d')
df = df[df.DT!='02-29'] #delete ferbruary 29th
df = df.groupby(by=['DT','STATE']).sum().reset_index()
df.DT = pd.to_datetime('2024-'+df.DT, format="%Y-%m-%d")
df = df.sort_values(by=['DT'])
df['DT_right'] = df.DT.dt.strftime('%m-%d')
dic = {df.DT_right.unique()[i]: i for i in np.arange(len(df.DT_right.unique()))}
df['DT_numeric'] = df.DT_right.apply(lambda x: dic[x])
date_dict = pd.Series(df['DT_numeric'].unique(), index=df['DT_right'].unique()).to_dict()
inverse_date_dict = {v: k for k, v in date_dict.items()}
#total deaths
df[death_column].sum()

22647923

In [ ]:
# How many deaths are there between 2000 and 2022?
death_column = 'DEATH_COUNT'
df = None
for state in states:
  data = pd.read_csv(path+f'sim/states_standardized/{state}_standardized_1996_2022.csv')
  data = data[['DT', death_column]]
  data['DT'] = pd.to_datetime(data['DT'])
  data = data[(data.DT.dt.year>=2000)&(data.DT.dt.year<=2022)]
  data['STATE'] = state
  if df is None:
    df = data
  else:
    df = pd.concat([df, data])
# I group all data from 2000-2019 in a single year that I set as
# 2024 only to be able to perform operations with it
df.DT = df.DT.dt.strftime('%m-%d')
df = df[df.DT!='02-29'] #delete ferbruary 29th
df = df.groupby(by=['DT','STATE']).sum().reset_index()
df.DT = pd.to_datetime('2024-'+df.DT, format="%Y-%m-%d")
df = df.sort_values(by=['DT'])
df['DT_right'] = df.DT.dt.strftime('%m-%d')
dic = {df.DT_right.unique()[i]: i for i in np.arange(len(df.DT_right.unique()))}
df['DT_numeric'] = df.DT_right.apply(lambda x: dic[x])
date_dict = pd.Series(df['DT_numeric'].unique(), index=df['DT_right'].unique()).to_dict()
inverse_date_dict = {v: k for k, v in date_dict.items()}
#total deaths
df[death_column].sum()

27575882